In [2]:
import pandas as pd
import time
import os
from rolling_lookahead_dt_pulp import rollo_oct_pulp

import cProfile

from helpers.helpers import preprocess_numerical, move_targets_to_front_and_rename, make_data_binary

target_label = "y"
depth_rolling_tree = 8
criterion_loss = "gini"

# Load your training and test datasets
train_data = pd.read_csv("datasets/adult/adult.data", sep=',', skipinitialspace=True, header=None) #32561 rows
test_data = pd.read_csv("datasets/adult/adult.test", sep=',', skipinitialspace=True, header=None) #16281 rows


#with open(f'{file_path}all_{dataset_name}.txt', 'w') as f:
    #f.write("Test adult dataset \n")

# Remove dots from the 'target' column
test_data[14] = test_data[14].astype(str).str.replace('.', '', regex=False)

len_train_data = len(train_data)

stacked = pd.concat([train_data, test_data ], ignore_index=False)
#print(stacked)


stacked = preprocess_numerical(stacked)
stacked = move_targets_to_front_and_rename(data= stacked, target_label=14)

unique_values = stacked ['y'].unique()
print(unique_values) # hier sieht man dann, dass es einen zusätzlichen punkt bei den target labels in adult.test gibt; also ohne löschen des punktes



stacked = make_data_binary(stacked)
print(stacked)




train_data_bin = stacked.iloc[:len_train_data] #[32561 rows x 125 columns]
test_data_bin = stacked.iloc[len_train_data:] #[16281 rows x 125 columns]

train_data=train_data_bin
test_data=test_data_bin

feature_columns = train_data.columns[1:] #assuming labels are in first column, ensured trough move_targets_to_front_and_rename()


def main(train_data, test_data, feature_columns, depth_rolling_tree, criterion_loss):
	result_dict_pulp =rollo_oct_pulp.run(train=train_data, test=test_data, target_label="y", features=feature_columns, depth=depth_rolling_tree, criterion=criterion_loss)
	
if __name__ == "__main__": 
	profiler = cProfile.Profile()
	profiler.enable()
	main(train_data, test_data, feature_columns, depth_rolling_tree, criterion_loss)
	profiler.disable()
	profiler.dump_stats("rollOCT_gini.out")

['<=50K' '>50K']
       y  1  2  3  4  5  6  7  8  9  ...  115  116  117  118  119  120  121  \
0      1  1  0  1  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
1      1  1  0  1  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
2      1  1  0  1  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
3      1  1  0  1  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
4      1  0  1  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
...   .. .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...   
16276  1  0  0  1  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
16277  1  1  0  0  1  0  1  0  0  0  ...    0    0    0    0    0    0    0   
16278  1  1  0  1  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
16279  1  1  0  1  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
16280  2  1  1  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   

       122  123  124  
0        1 